# Working with raw MRI data
Code adapted from: https://github.com/ismrmrd/ismrmrd-paper

This is an example Jupyter Notebook with simple code for reconstructing an MRI image from raw data in the ISMRMRD format, intended for educational purposes only. For details and more advanced methods check [Inati et al. ISMRM Raw Data Format: A Proposed Standard for MRI Raw Datasets](https://pmc.ncbi.nlm.nih.gov/articles/PMC4967038/) and resources provided by the [ISMRMRD development team](https://ismrmrd.github.io/apidocs/1.5.0/)

If you are working in Binder, you can start immediately with loading packages and reading in the pre-downloaded dataset. 

If you are working in your local python environment we first need to install the ismrmrd package. Do this run the following command in your terminal/command prompt. Not that this requires git to be installed on your system.

```bash
pip install git+https://github.com/ismrmrd/ismrmrd-python.git

## Downloading data

If you are working in binder, there is some data (testdata.h5) already downloaded. You can also download data and upload it here.

You can download data either by

### 1. Downloading from mridata.org
 
Go to mridata.org. Go to Data List. This script should work for any of the datasets from the NYU Machine Learning project or the Stanford 2D FSE project, e.g. the foot. You can download a dataset with sign up or login. Rename it to testdata.h5

### 2. Downloading the data used in the paper

Run command in terminal. Rename one of the files.

In [ ]:
#test

# working locally? install ismrmrd by running this in your terminal
#pip install git+https://github.com/ismrmrd/ismrmrd-python.git

# working in google colab or binder? Uncomment and run here
#!pip install git+https://github.com/ismrmrd/ismrmrd-python.git

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/ismrmrd/ismrmrd-python.git to c:\users\marle\appdata\local\temp\pip-req-build-gc5j3j_8
  Resolved https://github.com/ismrmrd/ismrmrd-python.git to commit b93bc1bf351bfd949a73343da3547be58922b819
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached h5py-3.14.0-cp311-cp311-win_amd64.whl (2.9 MB)
  Created wheel for ismrmrd: filename=ismrmrd-1.14.1-py3-none-any.whl size=31384 sha256=50b290292b28d7760a3485537dcb869761562b8f0075cad08ef511723913105d
  Stored in directory: C:\Users\marle\AppData\L

  Running command git clone --filter=blob:none --quiet https://github.com/ismrmrd/ismrmrd-python.git 'C:\Users\marle\AppData\Local\Temp\pip-req-build-gc5j3j_8'

[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
C:\Users\marle\AppData\Roaming\Python\Python311\site-packages\IPython\utils\_process_common.py:109: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  return out
C:\Users\marle\AppData\Roaming\Python\Python311\site-packages\IPython\utils\_process_common.py:109: ResourceWarning: unclosed file <_io.BufferedReader name=4>
  return out
C:\Users\marle\AppData\Roaming\Python\Python311\site-packages\IPython\utils\_process_common.py:109: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return out


In [24]:
import numpy as np
from numpy.fft import fftshift, ifftshift, fftn, ifftn
import ismrmrd
import ismrmrd.xsd
import h5py

In [31]:

#fname = 'knee'
fname = 'ismrmrd_data/ismrmrd_data/Bruker/bruker'

dset = ismrmrd.Dataset('%s.h5'%fname, 'dataset', create_if_needed=False)
header = ismrmrd.xsd.CreateFromDocument(dset.read_xml_header())
enc = header.encoding[0]

# Matrix size
eNx = enc.encodedSpace.matrixSize.x
eNy = enc.encodedSpace.matrixSize.y
eNz = enc.encodedSpace.matrixSize.z
rNx = enc.reconSpace.matrixSize.x
rNy = enc.reconSpace.matrixSize.y
rNz = enc.reconSpace.matrixSize.z

    # Number of Slices, Reps, Contrasts, etc.
if enc.encodingLimits.slice != None:
    nslices = enc.encodingLimits.slice.maximum + 1
else:
    nslices = 1

if enc.encodingLimits.repetition != None:
    nreps = enc.encodingLimits.repetition.maximum + 1
else:
    nreps = 1

if enc.encodingLimits.contrast != None:
    ncontrasts = enc.encodingLimits.contrast.maximum + 1
else:
    ncontrasts = 1

# the number of coils is optional, so get it from the first acquisition
acq = dset.read_acquisition(0)
ncoils = acq.active_channels


In [32]:
# Initialiaze a storage array
all_data = np.zeros((nreps, ncontrasts, nslices, ncoils, eNz, eNy, eNx), dtype=np.complex64)

# Loop through the rest of the acquisitions and stuff
for acqnum in range(dset.number_of_acquisitions()):
    acq = dset.read_acquisition(acqnum)

    # Ignore noise scans
    if acq.isFlagSet(ismrmrd.ACQ_IS_NOISE_MEASUREMENT):
        continue

    # Stuff into the buffer
    rep = acq.idx.repetition
    contrast = acq.idx.contrast
    slice = acq.idx.slice
    ky = acq.idx.kspace_encode_step_1
    kz = acq.idx.kspace_encode_step_2
    all_data[rep, contrast, slice, :, kz, ky, :] = acq.data

    
dset.close()


In [34]:
# we select one slice 
rep = 0
contrast = 0
slice = 10
slice = 0 #set this anywhere between 0 and nSlices-1
coil = 0
kz = 0 #because we used slice encoding

slice_data = all_data[rep,contrast,slice,coil,kz,:,:]   

In [35]:
fname = 'k_slice_kiwi.h5'

# Stick the array into the hdf5 file
with h5py.File(fname, 'w') as f:
    f.create_dataset('/K_slice_real', data=np.real(slice_data), dtype='float64')
    f.create_dataset('/K_slice_imag', data=np.imag(slice_data), dtype='float64')

f.close()
  